statistics-resampling online 
============

Interact with Octave and the statistics-resampling package online in Jupyter Notebook with Binder. All commands are interpreted by Octave.  Since this is a [MetaKernel](https://github.com/Calysto/metakernel), a standard set of magics are available.  Help on commands is available using the `%help` magic or using `?` with a command.

Follow the links in the 'Quick start' section of the [README page](https://gnu-octave.github.io/statistics-resampling/readme.html), or on the [Function reference page](https://gnu-octave.github.io/statistics-resampling/function_reference), to find some examples relevant to you that can be copy-and-pasted below and updated with your own data. *Ensure that code in each cell generates no more than one plot to the same figure  (unless using subplot) - replacing figures or plots does not display properly in gnuplot*. 
    
To see a list of other Octave packages available, use the command `pkg list`. To load `<package-name>`, use the command `pkg load <package-name>`, being sure to replace `<package-name>` with the name of the package you wish to load. Visit this [webpage](https://gnu-octave.github.io/packages/) to see what packages are available for GNU Octave.
    
You could consider forking and customizing the statistics-resampling-online [GitHub repository](https://github.com/acpennlab/statistics-resampling-online) to document and publish your data analysis project.

In [ ]:
# YOU MUST RUN THIS CELL FIRST TO INSTALL AND LOAD THE STATISTICS-RESAMPLING PACKAGE
fprintf('Any gnuplot warnings above are not a concern since our plotting requirements are very basic.\n');
pkg install -forge statistics-resampling;
pkg load statistics-resampling;
computer
ver

In [ ]:
# ENTER YOUR DATA AND CODE BELOW IN THIS CELL ONWARDS

